## RAG

In [1]:
%pip install chromadb
%pip install PyPDF2
import chromadb
import os
client = chromadb.PersistentClient(path="./dbstore")


def pdf_extraction_alg(uploaded_file):

    root_directory_path = os.getcwd()
    unique_file_name = os.path.join(root_directory_path, uploaded_file.name)

    with open(unique_file_name, 'wb') as destination_file:
        for chunk in uploaded_file.chunks():
            destination_file.write(chunk)

    pdf_text = []

    with open(unique_file_name, 'rb') as pdf:
        reader = PyPDF2.PdfReader(pdf, strict=False)

        for page in reader.pages:
            content = page.extract_text()
            pdf_text.append(content)

    os.remove(unique_file_name)

    output_string = ''

    for item in pdf_text:
        output_string += item
    return output_string

collection = client.get_or_create_collection("sample-collection")


[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:

collection.add(
    documents=["This is document1", "This is document2"], # we embed for you, or bring your own
    metadatas=[{"source": "notion", "course-id":"20CS2056"}, {"source": "google-docs", "course-id":"20CS2065"}], # filter on arbitrary metadata!
    ids=["doc1", "doc2"], # must be unique for each doc 
)


Insert of existing embedding ID: doc1
Insert of existing embedding ID: doc2
Add of existing embedding ID: doc1
Add of existing embedding ID: doc2


In [4]:

results = collection.query(
    query_texts=["This is a query document"],
    n_results=1,
    where={"course-id": "notion"}, # optional filter
    # where_document={"$contains":"search_string"}  # optional filter
)  

In [5]:
for i in results:
    print(i, results[i])

ids [[]]
distances [[]]
metadatas [[]]
embeddings None
documents [[]]
uris None
data None


In [20]:
with open("Exception Handling in C# (1).pdf","r") as f:
    pdf_extraction_alg("Exception Handling in C# (1).pdf")

AttributeError: 'str' object has no attribute 'name'